In [1]:
! pip install kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 915MB/s]


In [5]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [6]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [11]:
twitter_dataset = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [13]:
twitter_data.shape


(1599999, 6)

In [14]:
twitter_dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [15]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_dataset = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')

In [16]:
twitter_dataset.shape


(1600000, 6)

In [17]:
twitter_dataset.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
twitter_dataset.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [20]:
twitter_dataset['target'].value_counts()

,count
target,
0,800000
4,800000


In [21]:
twitter_dataset.replace({'target': {4:1}},inplace=True)

In [22]:
twitter_dataset['target'].value_counts()

,count
target,
0,800000
1,800000


In [24]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [PorterStemmer().stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [25]:
twitter_dataset['text'] = twitter_dataset['text'].apply(stemming)

In [26]:
twitter_dataset.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass behav mad see


In [28]:
print(twitter_dataset['text'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: text, Length: 1600000, dtype: object


In [29]:
print(twitter_dataset['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [34]:
X = twitter_dataset['text'].values
Y = twitter_dataset['target'].values

In [35]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [36]:
print(Y)

[0 0 0 ... 1 1 1]


In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [38]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [40]:
print(X_train.shape)

(1280000,)


In [41]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [42]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [43]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 532699)>
  Coords	Values
  (0, 126118)	0.37484032460318173
  (0, 213554)	0.5289928671316323
  (0, 271144)	0.42037613603065954
  (0, 409438)	0.35861719248416485
  (0, 504044)	0.2725664180116586
  (0, 511398)	0.4472655127083222
  (1, 185304)	1.0
  (2, 90030)	0.31341180319788786
  (2, 126118)	0.45825085123736603
  (2, 143635)	0.18906085394224742
  (2, 149331)	0.29041017783193285
  (2, 152669)	0.20261379420870737
  (2, 173898)	0.1880703874098935
  (2, 205312)	0.16175394132247145
  (2, 307827)	0.2410455646561603
  (2, 333015)	0.16780678648815192
  (2, 468994)	0.32099337313155857
  (2, 470034)	0.18720443014780438
  (2, 472107)	0.15171423197968206
  (2, 500375)	0.3297483694639193
  (2, 511810)	0.33617620658621283
  (3, 65151)	0.5205345321237522
  (3, 175126)	0.27847482359309217
  (3, 183099)	0.44549570924150195
  (3, 198842)	0.3743117172596608
  :	:
  (1279996, 336028)	0.17877262371579808

In [44]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2363913 stored elements and shape (320000, 532699)>
  Coords	Values
  (0, 17371)	0.1720259377078855
  (0, 35963)	0.16255032347453868
  (0, 78282)	0.2682586678919192
  (0, 122384)	0.36470114019452204
  (0, 152729)	0.2555879644965987
  (0, 159349)	0.2366728795611819
  (0, 197646)	0.28105531331632916
  (0, 312772)	0.45307191487898857
  (0, 322141)	0.1784504098473567
  (0, 448260)	0.22002407032145394
  (0, 460402)	0.3491793745175741
  (0, 472107)	0.31471678837594874
  (0, 485830)	0.17939145201290535
  (1, 7432)	0.2554738596093585
  (1, 17371)	0.17542242601230812
  (1, 167742)	0.4782713992526613
  (1, 250883)	0.3354843401454554
  (1, 296228)	0.23864015910121067
  (1, 402004)	0.3950023328793501
  (1, 406721)	0.5548902184495058
  (1, 422904)	0.20442035768338634
  (2, 25933)	0.35382372494717845
  (2, 39714)	0.38049318170650237
  (2, 103267)	0.36475227351737843
  (2, 211386)	0.5856420797439991
  :	:
  (319994, 512231)	0.2352338329336

In [45]:
model = LogisticRegression(max_iter=1000)

In [56]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [47]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [48]:
print('Accuracy score on the training data :',training_data_accuracy)

Accuracy score on the training data : 0.8047734375


In [49]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [50]:
print('Accuracy score on the test data : ', test_data_accuracy)

Accuracy score on the test data :  0.777475


In [51]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [52]:
print('Accuracy score on the test data : ', test_data_accuracy)

Accuracy score on the test data :  0.777475


In [54]:
import pickle

In [57]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [64]:
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [65]:
X_new = X_test[200]
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

1
[1]
Positive Tweet


In [63]:
# Example of making predictions with the loaded model

# You can replace this with any new text you want to classify
new_text = "This is a great day!"

# Preprocess the new text using the same stemming function and vectorizer
stemmed_new_text = stemming(new_text)
vectorized_new_text = vectorizer.transform([stemmed_new_text])

# Make a prediction
prediction = loaded_model.predict(vectorized_new_text)

# Print the prediction
if prediction[0] == 1:
  print("The sentiment of the text is Positive")
else:
  print("The sentiment of the text is Negative")

The sentiment of the text is Positive
